# 1. Initialisation

Importations

In [108]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg")  # Ou "Qt5Agg"
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random

Paramètres du modèle

In [109]:
window_size = 7
distance = 5
angle = 0.2
dt = 1
noise = 0.1
resolution = 0.002
duration = 1000
beam_radius = 0
a = 0.3  # Facteur d'expansion de la spirale
b = 0.1  # Facteur de croissance du rayon

Initialisation des positions

In [ ]:
target_position = np.array([np.random.uniform(window_size/4, 3*window_size/4), np.random.uniform(window_size/4, 3*window_size/4)])
target_speed = np.array([np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1)])
sensor_position = np.array([window_size/2, window_size/2])
tracking_position = np.copy(sensor_position)

target_history = [target_position.copy()]
tracking_history = [tracking_position.copy()]

is_target_found = False

search_path = [(0, 0)] * duration

Initialisation du plot

In [111]:
fig, ax = plt.subplots()
ax.set_xlim(0, window_size)
ax.set_ylim(0, window_size)
target_line, = ax.plot([], [], 'r--', alpha=0.5, label="Trajectoire cible")
tracking_line, = ax.plot([], [], 'b--', alpha=0.5, label="Trajectoire suivi")
sensor_dot, = ax.plot([], [], 'go', markersize=8, label="Capteur")
target_dot, = ax.plot([], [], 'ro', markersize=8, label="Cible")
tracking_dot, = ax.plot([], [], 'bo', markersize=8, label="Tracking")
ax.legend()
ax.set_title("Simulation Pointing, Acquisition, and Tracking (PAT)")

Text(0.5, 1.0, 'Simulation Pointing, Acquisition, and Tracking (PAT)')

# 2. PAT

A. Outils

In [112]:
def calculate_radius(distance, angle):
    """Calcule le rayon d'un faisceau à une certaine distance de la source"""
    radius = distance * np.tan(angle)
    return radius


def detect_target(beam_radius):
    """Teste si la cible est dans le rayon du faisceau"""
    distance = np.linalg.norm(target_position - tracking_position)
    return distance <= beam_radius

B. Calcul du parcours du faisceau

In [113]:
def make_square_spiral_path(frame, beam_radius):
    """Calcule les positions successives d'un faisceau parcourant une spirale carrée"""
    global search_path

    point = tracking_position.copy()
    rank = frame
    search_path[rank] = point.copy()
    step = 0.5*beam_radius

    side_length = 1
    direction = 1
    while rank < duration:
        for i in range(side_length):
            rank += 1
            if rank >= duration:
                return           
            point[0] = point[0] + direction * step
            search_path[rank] = point.copy()
        for i in range(side_length):
            rank += 1
            if rank >= duration:
                return
            point[1] = point[1] + direction * step
            search_path[rank] = point.copy()
        direction *= -1
        side_length += 1


def make_circle_spiral_path(frame, beam_radius):
    """Calcule les positions successives d'un faisceau parcourant une spirale ronde"""
    global search_path

    point = tracking_position.copy()
    rank = frame
    search_path[rank] = point.copy()

    while rank < duration - 1:
        theta = a * rank  # Angle qui tourne
        r = b * rank      # Rayon qui grandit
        x = 50 + r * np.cos(theta)
        y = 50 + r * np.sin(theta)
        search_path[rank] = np.array([x, y])
        rank += 1

C. Calcul des déplacements des éléments

In [114]:
def move_target():
    """Met à jour les position et vitesse de la cible"""
    global target_position
    global target_speed

    x, y = target_position
    vx, vy = target_speed

    new_x = x + vx*dt + random.gauss(0, noise*vx*dt)
    new_y = y + vy*dt + random.gauss(0, noise*vy*dt)
    new_vx = (new_x - x) / dt
    new_vy = (new_y - y) / dt
    
    target_position = np.array([new_x, new_y])
    target_speed = np.array([new_vx, new_vy])
    target_position = np.clip(target_position, 0, window_size)
    target_speed = np.clip(target_speed, 0, window_size)

    # Limiter la taille de l'historique pour éviter une surcharge mémoire
    if len(target_history) > 50:
        target_history.pop(0)
    target_history.append(target_position.copy())


def search_target(frame):
    """Met à jour la position d'un faisceau cherchant la cible"""
    global tracking_position, is_target_found, beam_radius
    tracking_position = np.clip(search_path[frame], 0, window_size)

    if len(tracking_history) > 50:
        tracking_history.pop(0)
    tracking_history.append(tracking_position.copy())

    if detect_target(beam_radius):
        if beam_radius <= resolution:
            is_target_found = True
        else:
            beam_radius /= 2
            make_square_spiral_path(frame, beam_radius)
            

def track_target(frame):
    """Met à jour la position d'un faisceau traquant une cible qu'il a déjà trouvé"""
    global is_target_found
    if not detect_target(beam_radius):
        is_target_found = False
        make_square_spiral_path(frame, beam_radius)    


D. Mise à jour de l'animation

In [115]:
def update(frame):
    """Détermine l'état de la frame suivante"""
    global tracking_position, target_position

    move_target()
    if is_target_found:
        track_target(frame)        
    else :
        search_target(frame)

    # Mise à jour des données des objets graphiques
    target_x, target_y = zip(*target_history)
    tracking_x, tracking_y = zip(*tracking_history)

    target_line.set_data(target_x, target_y)
    tracking_line.set_data(tracking_x, tracking_y)
    target_dot.set_data(target_position[0], target_position[1])
    sensor_dot.set_data(sensor_position[0], sensor_position[1])
    tracking_dot.set_data(tracking_position[0], tracking_position[1])

    return target_line, target_dot, sensor_dot, tracking_line, tracking_dot,

E. Exécution

In [116]:

beam_radius = calculate_radius(distance, angle)
make_square_spiral_path(0, beam_radius)

ani = animation.FuncAnimation(fig, update, frames=1000, interval=100, blit=True)
plt.show()